<a href="https://colab.research.google.com/github/dhanesh123in/blob/main/deeplearning/LayoutLMv2/FUNSD/Fine_tuning_LayoutLMv2ForTokenClassification_on_FUNSD-EL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q datasets seqeval
!pip install -q pyyaml==5.1
!pip install -q torch==1.8.0+cu101 torchvision==0.9.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!python -m pip install -q 'git+https://github.com/facebookresearch/detectron2.git'

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 67 kB 3.1 MB/s 
     |████████████████████████████████| 895 kB 15.8 MB/s 
     |████████████████████████████████| 596 kB 11.0 MB/s 
     |████████████████████████████████| 6.5 MB 32.5 MB/s 
     |████████████████████████████████| 311 kB 13.2 MB/s 
     |████████████████████████████████| 43 kB 1.7 MB/s 
     |████████████████████████████████| 1.1 MB 45.8 MB/s 
     |████████████████████████████████| 212 kB 54.1 MB/s 
     |████████████████████████████████| 134 kB 54.3 MB/s 
     |████████████████████████████████| 271 kB 56.1 MB/s 
     |████████████████████████████████| 94 kB 2.7 MB/s 
     |████████████████████████████████| 144 kB 49.1 MB/s 
     |████████████████████████████████| 274 kB 15.3 MB/s 
     |████████████████████████████████| 763.5 MB 16 kB/s 
     |████████████████████████████████| 17.3 MB 898 kB/s 
ERROR: pip

In [3]:
!curl -O https://guillaumejaume.github.io/FUNSD/dataset.zip
!unzip dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.0M  100 16.0M    0     0  26.5M      0 --:--:-- --:--:-- --:--:-- 26.5M
Archive:  dataset.zip
   creating: dataset/
   creating: dataset/training_data/
  inflating: dataset/training_data/.DS_Store  
   creating: __MACOSX/
   creating: __MACOSX/dataset/
   creating: __MACOSX/dataset/training_data/
  inflating: __MACOSX/dataset/training_data/._.DS_Store  
   creating: dataset/training_data/images/
  inflating: dataset/training_data/images/92091873.png  
   creating: __MACOSX/dataset/training_data/images/
  inflating: __MACOSX/dataset/training_data/images/._92091873.png  
  inflating: dataset/training_data/images/91939637.png  
  inflating: __MACOSX/dataset/training_data/images/._91939637.png  
  inflating: dataset/training_data/images/87533049.png  
  inflating: __MACOSX/dataset/training_data/images/._87533049.png  
  inflati

In [4]:
import os
import json
import re
import pandas as pd
import itertools
import functools
print(len(os.listdir("dataset/training_data/images")))


train_data_dir="dataset/training_data/"
test_data_dir="dataset/testing_data/"

df_tmplt={'fname':[],'image_path':[], "id":[], "words":[], "bboxes":[], 'labels':[]}
df_dict={}

def list_append(l1,l2,i):
    lr=l1
    #print(l1,l2)
    if l2:
        if l2[0]==i:
            lr.extend([l2[1]])
    return lr

def traverse_accumulate(i,ids_c,ids_v):
    ids_r=[]
    
    if i in ids_v:
        return ids_v,ids_r
    else:
        ids_r=[i]
        ids_v.add(i)
        ids_vn=ids_v
        
        if not ids_c[i]:
            return ids_vn,ids_r
        
        for ii in ids_c[i]:
            ids_vn, ids_rn=traverse_accumulate(ii,ids_c,ids_v)
            ids_r.extend(ids_rn)
        return ids_vn,ids_r

def datagen(data_dir):
    for f in os.listdir(data_dir+"annotations"):
        fname=re.sub(".json","",f)
        df_dict=df_tmplt.copy()
        df_dict['fname'].append(fname)
        df_dict['image_path'].append(data_dir+"images/"+fname+".png")
        with open(data_dir+"annotations/"+f) as fa:
            ann=json.load(fa)
            ids=[]
            words=[]
            bboxes=[]
            linkings=[]
            labels=[]
            for frm in ann['form']:
                id=frm['id']
                l=frm['linking']
                ids.append(id)
                linkings.append(l)
                wd=[]
                bb=[]
                lb=[]
                for w in frm['words']:
                    wd.append(w['text'])
                    bb.append(w['box'])
                    lb.append(frm['label'])
                words.append(wd)
                bboxes.append(bb)
                labels.append(lb)       

            words_r=[]
            bboxes_r=[]
            labels_r=[]
            ids_visited=set()
            ids_combined={}
            for i, li in enumerate(linkings):
                ids_combined[i]=list(functools.reduce(functools.partial(list_append,i=i),li,[]))

            cnt=0
            for i in ids_combined.keys():
                ids_visited,links=traverse_accumulate(i,ids_combined,ids_visited)

                for l in links:
                    words_r.extend(words[l])
                    bboxes_r.extend(bboxes[l])
#                     if (len(links)>1):
                    labels_r.extend([str(cnt).zfill(3)+"_"+lb if lb in ['question','answer'] else lb for lb in labels[l]])
#                     else:
#                         labels_r.extend([lb for lb in labels[l]])
                if links:
                    cnt=cnt+1

            df_dict['id'].append(ids)
            df_dict['words'].append(words_r)
            df_dict['bboxes'].append(bboxes_r)
            df_dict['labels'].append(labels_r)
            #df_dict['labels']=pd.Categorical({v for k,v in enumerate(labels_r)})

    df=pd.DataFrame.from_dict(df_dict)
    return df

df_train=datagen(train_data_dir)
df_test=datagen(test_data_dir)

149


In [5]:
labels=set()
for i,row in df_train.iterrows():
    labels=labels.union(set(row['labels']))

print(len(labels))
    
for i,row in df_test.iterrows():
    labels=labels.union(set(row['labels']))
labels=sorted(labels)
print(len(labels))

label2id={v:k for k,v in enumerate(labels)}
print(label2id)

id2label={k:v for k,v in enumerate(labels)}
print(id2label)

df_train['labels']=df_train['labels'].apply(lambda x: [label2id[xx] for xx in x])
df_test['labels']=df_test['labels'].apply(lambda x: [label2id[xx] for xx in x])

180
184
{'000_answer': 0, '000_question': 1, '001_answer': 2, '001_question': 3, '002_answer': 4, '002_question': 5, '003_answer': 6, '003_question': 7, '004_answer': 8, '004_question': 9, '005_answer': 10, '005_question': 11, '006_answer': 12, '006_question': 13, '007_answer': 14, '007_question': 15, '008_answer': 16, '008_question': 17, '009_answer': 18, '009_question': 19, '010_answer': 20, '010_question': 21, '011_answer': 22, '011_question': 23, '012_answer': 24, '012_question': 25, '013_answer': 26, '013_question': 27, '014_answer': 28, '014_question': 29, '015_answer': 30, '015_question': 31, '016_answer': 32, '016_question': 33, '017_answer': 34, '017_question': 35, '018_answer': 36, '018_question': 37, '019_answer': 38, '019_question': 39, '020_answer': 40, '020_question': 41, '021_answer': 42, '021_question': 43, '022_answer': 44, '022_question': 45, '023_answer': 46, '023_question': 47, '024_answer': 48, '024_question': 49, '025_answer': 50, '025_question': 51, '026_answer':

In [6]:
from datasets import Dataset, DatasetDict

datasets = DatasetDict({"train": Dataset.from_pandas(df_train, split="train"), 
                        "test": Dataset.from_pandas(df_train, split="test")})

In [7]:
datasets

DatasetDict({
    train: Dataset({
        features: ['fname', 'image_path', 'id', 'words', 'bboxes', 'labels'],
        num_rows: 149
    })
    test: Dataset({
        features: ['fname', 'image_path', 'id', 'words', 'bboxes', 'labels'],
        num_rows: 149
    })
})

In [8]:
a=datasets['train'][100]['labels']
b=datasets['train'][100]['words']

print(datasets['train'][100]['fname'])
res = "\n".join("{} {}".format(id2label[x], y) for x, y in zip(a, b))
print(res)

89386032
header EPL
001_answer Multiple
002_answer 1419A
003_question Species
003_answer Mice
other 89386032
header EXPERIMENTAL
header PATHOLOGY
header LABORATORIES,
header INC.
header QUALITY
header ASSURANCE
header REPORT
header CERTIFICATION
007_question Client
007_question Name
007_answer Borriston
007_answer Laboratories,
007_answer Inc.
008_question Study
008_question Director
008_answer Dr.
008_answer William
008_answer O.
008_answer Iverson
008_answer Pathologist
008_answer Dr.
008_answer Charley
008_answer E.
008_answer Gilmore
009_question Study
009_question Title
009_answer Bioassay
009_answer of
009_answer Cigarette
009_answer Smoke
009_answer Condensates
009_answer for
009_answer potential
009_answer Tumorigenic
009_answer Activity
009_answer on
009_answer Mouse
009_answer Skin
009_answer (Revised
009_answer Pathology
009_answer Report)
010_question Test
010_question Article
011_question Client
011_question Study
011_question No.
other A11
other parts
other of
other the
o

In [9]:
from PIL import Image
from transformers import LayoutLMv2Processor
from datasets import Features, Sequence, ClassLabel, Value, Array2D, Array3D

processor = LayoutLMv2Processor.from_pretrained("microsoft/layoutlmv2-base-uncased", revision="no_ocr")

# we need to define custom features
features = Features({
    'image': Array3D(dtype="int64", shape=(3, 224, 224)),
    'input_ids': Sequence(feature=Value(dtype='int64')),
    'attention_mask': Sequence(Value(dtype='int64')),
    'token_type_ids': Sequence(Value(dtype='int64')),
    'bbox': Array2D(dtype="int64", shape=(512, 4)),
    'labels': Sequence(ClassLabel(names=labels)),
})

def preprocess_data(examples):
  images = [Image.open(path).convert("RGB") for path in examples['image_path']]
  words = examples['words']
  boxes = examples['bboxes']
  word_labels = examples['labels']
  
  encoded_inputs = processor(images, words, boxes=boxes, word_labels=word_labels,
                             padding="max_length", truncation=True)
  
  return encoded_inputs

train_dataset = datasets['train'].map(preprocess_data, batched=True, remove_columns=datasets['train'].column_names,
                                      features=features)
test_dataset = datasets['test'].map(preprocess_data, batched=True, remove_columns=datasets['test'].column_names,
                                      features=features)

Downloading:   0%|          | 0.00/136 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/707 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
train_dataset.set_format(type="torch", device="cuda")
test_dataset.set_format(type="torch", device="cuda")

In [11]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2)

In [12]:
batch = next(iter(train_dataloader))

for k,v in batch.items():
  print(k, v.shape)

image torch.Size([4, 3, 224, 224])
input_ids torch.Size([4, 512])
attention_mask torch.Size([4, 512])
token_type_ids torch.Size([4, 512])
bbox torch.Size([4, 512, 4])
labels torch.Size([4, 512])


In [21]:
from transformers import LayoutLMv2ForTokenClassification, AdamW
import torch
from tqdm.notebook import tqdm

model = LayoutLMv2ForTokenClassification.from_pretrained('microsoft/layoutlmv2-base-uncased',
                                                          num_labels=len(labels))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-5)

global_step = 0
num_train_epochs = 10
t_total = len(train_dataloader) * num_train_epochs # total number of training steps 

#put the model in training mode
model.train() 
for epoch in range(num_train_epochs):  
   print("Epoch:", epoch)
   for batch in tqdm(train_dataloader):
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(**batch) 
        loss = outputs.loss
        
        # print loss every 100 steps
        if global_step % 100 == 0:
          print(f"Loss after {global_step} steps: {loss.item()}")

        loss.backward()
        optimizer.step()
        global_step += 1

Some weights of the model checkpoint at microsoft/layoutlmv2-base-uncased were not used when initializing LayoutLMv2ForTokenClassification: ['layoutlmv2.visual.backbone.bottom_up.res4.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.2.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.22.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.7.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.8.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res2.2.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.13.conv2.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res4.22.conv3.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res3.1.conv1.norm.num_batches_tracked', 'layoutlmv2.visual.backbone.bottom_up.res5.0.conv3.norm.num_batches_tracked'

RuntimeError: ignored

In [14]:
from datasets import load_metric

metric = load_metric("seqeval")

# put model in evaluation mode
model.eval()
for batch in tqdm(test_dataloader, desc="Evaluating"):
    with torch.no_grad():
        input_ids = batch['input_ids'].to(device)
        bbox = batch['bbox'].to(device)
        image = batch['image'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        # forward pass
        outputs = model(input_ids=input_ids, bbox=bbox, image=image, attention_mask=attention_mask, 
                        token_type_ids=token_type_ids, labels=labels)
        
        # predictions
        predictions = outputs.logits.argmax(dim=2)

        # Remove ignored index (special tokens)
        true_predictions = [
            [id2label[p.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [id2label[l.item()] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]

        metric.add_batch(predictions=true_predictions, references=true_labels)

final_score = metric.compute()
print(final_score)

Downloading:   0%|          | 0.00/2.47k [00:00<?, ?B/s]

Evaluating:   0%|          | 0/75 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 000_answer seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 001_question seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 002_question seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: 002_answer seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: header seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/

{'00_answer': {'precision': 0.7017543859649122, 'recall': 0.6837606837606838, 'f1': 0.6926406926406926, 'number': 117}, '00_question': {'precision': 0.38181818181818183, 'recall': 0.860655737704918, 'f1': 0.5289672544080605, 'number': 122}, '01_answer': {'precision': 0.358974358974359, 'recall': 0.358974358974359, 'f1': 0.358974358974359, 'number': 117}, '01_question': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 129}, '02_answer': {'precision': 0.2950310559006211, 'recall': 0.7089552238805971, 'f1': 0.4166666666666667, 'number': 134}, '02_question': {'precision': 0.25517241379310346, 'recall': 0.27611940298507465, 'f1': 0.26523297491039427, 'number': 134}, '03_answer': {'precision': 0.23076923076923078, 'recall': 0.08571428571428572, 'f1': 0.12500000000000003, 'number': 105}, '03_question': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 116}, '04_answer': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 96}, '04_question': {'precision': 0.0, 'recall': 0.0, 'f1'

In [18]:
for kk in ['overall_precision', 'overall_recall', 'overall_f1', 'overall_accuracy']:
  print(kk,final_score[kk])

overall_precision 0.15269538302811453
overall_recall 0.10044112656939261
overall_f1 0.12117490533210522
overall_accuracy 0.2942201834862385
